In [1]:
import os

In [3]:
os.chdir("../") # this means 'go up one folder'
%pwd

'/home/keenanandrew/krish-naik/End-to-end-ML-project'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


# takes info from config.yaml
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path